# KINTSUGI

## In the following notebook you will use the parameters found in A_Evaluation&Testing.ipynb to correct and stitch entire datasets.

### 1. Import packages. *This must be done every time the notebook is started or restarted.¶

Import these packages. Run cells using Ctrl+Enter.

In [1]:
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import gc
import os as os
from tqdm import trange
import cupy as cp
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog
import pandas as pd
from m2stitch import stitch_images
from basicpy import BaSiC, metrics
from glob import glob
from skimage.io.collection import alphanumeric_key
import shutil
from hyperactive import Hyperactive
import numpy as np
from skimage.io import imread 
from skimage.io import imsave
from skimage import io
from skimage import util
from skimage import exposure
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import jax
import pickle
from itertools import chain, repeat
import subprocess
from datetime import datetime
import imagej, scyjava
current_dateTime = datetime.now()

jax.config.update('jax_platform_name', 'cpu')

### 2. Define directory paths. *This must be done every time the notebook is started or restarted.

Below are three ways to get the required paths to input, output, and meta folders.  The first is where they can be entered manually.  The second-fourth simply asks for the location of each.  The third reads from the project_data.txt file created in the first notebook and appends the variables to the global variable list.

In each of these methods, a text file to store correction parameters is defined.

Choose only one method: A, B, or C.

### 2.1 Method A

In [2]:
base_dir = "C:/Users/smith6jt/KINTSUGI"
image_dir ="C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_raw"
stitch_dir = "C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_BaSiC_Stitched"
meta_dir = "C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_meta"
project_file = os.path.join(meta_dir, "project_data.txt")
print(f"Image folder is {image_dir}.")
print(f"Stitching folder is {stitch_dir}.")
print(f"Meta folder is {meta_dir}.")

Image folder is C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_raw.
Stitching folder is C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_BaSiC_Stitched.
Meta folder is C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_meta.


### 2.2 Method B

In [ ]:
# Enter image input directory.
image_dir = filedialog.askdirectory()
print(f"Image folder is {image_dir}.")

In [ ]:
# Enter stitching output directory.
stitch_dir = filedialog.askdirectory()
project_file = os.path.join(meta_dir, "project_data.txt")
print(f"Stitching folder is {stitch_dir}.")

In [ ]:
# Enter the metadata directory.
stitch_dir = filedialog.askdirectory()
project_file = os.path.join(meta_dir, "project_data.txt")
print(f"Meta folder is {meta_dir}.")

### 2.3 Method C

If folder names were saved previously, import them here.

In [ ]:
# Enter the metadata directory.
meta_dir = filedialog.askdirectory()
project_filename = os.path.join(meta_dir, "project_data.txt")
with open(project_file, 'r') as file:
        for line in file:
            try:
                line = line.strip()
                var_name, var_value = line.split('=')
                if var_name in globals():
                    print(f"{var_name} is {var_value}")
            except (ValueError):
                pass


### 3. Stitching and Illumination Correction Functions

### 3.1 Stitching Function

The following cell defines the function that is called from the apply_basic function to stitch the corrected tiles.  It creates the model from the middle z-plane images of the first channel, and then applies the model to the rest of the images for all channels in the cycle.

In [4]:
def stitch(images_transformed, zplanes, dest, dest_1, channels, zplanes_n, pou, rows, cols, overlap_percentage):
    
    z = str(zplanes)
    ch = str(channels)
    pkl_dest = os.path.join(dest, "result_df.pkl")
    
    if zplanes == zplanes_n//2 and channels == 1:
        # print(f"Start Stitching Z0{z.zfill(2)}_CH{ch}")
        if os.path.exists(pkl_dest):
            result_df = pd.read_pickle(pkl_dest)
        else:
            result_df, _ = stitch_images(images_transformed, rows, cols, initial_ncc_threshold=0.0, overlap_percentage=overlap_percentage, pou=pou, max_cores=10)
            result_df.to_pickle(pkl_dest)
            cp._default_memory_pool.free_all_blocks()
        
    else:
        # print(f"Start Stitching Z0{z.zfill(2)}_CH{ch}")
        if os.path.exists(os.path.join(dest_1, "result_df.pkl")):
            result_df = pd.read_pickle(os.path.join(dest_1, "result_df.pkl"))

        else:
            print("Run registration channel to produce a stitching model.")

    result_df["y_pos2"] = result_df["y_pos"] - result_df["y_pos"].min()
    result_df["x_pos2"] = result_df["x_pos"] - result_df["x_pos"].min()
    
    size_y = images_transformed.shape[1]
    size_x = images_transformed.shape[2]
    
    stitched_image_size = (
        result_df["y_pos2"].max() + size_y,
        result_df["x_pos2"].max() + size_x,
    )
    stitched_image = np.zeros_like(images_transformed, shape=stitched_image_size)
    
    for i, row in result_df.iterrows():
        stitched_image[
            row["y_pos2"] : row["y_pos2"] + size_y,
            row["x_pos2"] : row["x_pos2"] + size_x,
        ] = images_transformed[i]

    result_image_file_path = os.path.join(dest,f"{z.zfill(2)}.tif") 
    imsave(result_image_file_path, stitched_image, check_contrast=False)
    
    # print(f"Saved to {result_image_file_path}")


### 3.2 Illumination Correction Function

The following cell contains the function that will apply the BaSiC correction to your data.  Make sure the number of leading zero digits in your cycle folder names is correct for 'zeros' and the filename pattern is correct for 'im_raw'.  This function runs BaSiC for the middle z-plane images and then reuses the correction model for the rest of the channel's images.  It then calls the stitching function.

In [ ]:
def apply_basic(image_dir, stitch_dir, zplanes, cycles, channels, zplanes_n, pou, rows, cols, overlap_percentage):
    
    max_basic_workers = os.cpu_count()
    zeros = 3
    quant = 0.95
    filename_pattern = f'1_000??_Z0{str(zplanes).zfill(2)}_CH{str(channels)}.tif'
    
    dest = os.path.join(stitch_dir, f"cyc{str(cycles).zfill(2)}", f"CH{str(channels)}")
    os.makedirs(dest, exist_ok=True)
    dest_1 = os.path.join(stitch_dir, f"cyc{str(cycles).zfill(2)}", "CH1")
    
    im_raw = sorted(glob(os.path.join(image_dir, f'cyc{str(cycles).zfill(zeros)}', filename_pattern)), key=alphanumeric_key)
    im = io.imread_collection(im_raw)
    im_array = np.asarray(im)
    if np.min(im_array) != 0:
        im_array = im_array - np.min(im_array)
    # im_i_max, im_i_min = np.max(im_array), np.min(im_array)
    # print(f"Cycle {cycles} channel {channels} z-plane {zplanes} min, max is {im_i_min}, {im_i_max} range is {im_i_max - im_i_min}") 
    # im_array = exposure.rescale_intensity(im_array, out_range=(0, 1))
    
    if zplanes == zplanes_n//2:

        # print(f"Start Illumination Correction cyc{str(cycles).zfill(2)} Z0{str(zplanes).zfill(2)}_CH{str(channels)}")
        basic = BaSiC(max_workers=max_basic_workers, max_reweight_iterations=30, max_reweight_iterations_baseline=40)
        basic.autotune(im_array, random_state=0, early_stop=False, histogram_qmin=1-quant, histogram_qmax=quant, search_space={"smoothness_flatfield": list(np.logspace(-1, 1, 10))}, init_params={"smoothness_flatfield": 0.1})
        # basic=BaSiC.load_model(os.path.join(dest, "basic_first"))

        images_transformed_ff = basic.fit_transform(im_array)
        basic.save_model(os.path.join(dest, "basic_first"), overwrite=True)
        # ratio_1 = 1 / 1
        # print(f"Image quantile: {np.quantile(im_array, quant)} flatfield quantile: {np.quantile(basic.flatfield, quant)}")
        # print(f"Cycle {cycles} channel {channels} z-plane {zplanes} first ratio: {ratio_1}")      
        # images_transformed_ff = exposure.rescale_intensity(images_transformed_ff, out_range=(0, 1)) 
              
        basic_2 = BaSiC(fitting_mode='approximate', get_darkfield=True, sort_intensity=True, max_workers=max_basic_workers)
        basic_2.autotune(images_transformed_ff, random_state=0, n_iter=50, early_stop_n_iter_no_change=8, histogram_bins=100, histogram_qmax=0.6, 
                        search_space={"smoothness_flatfield": [0.1],
                                    "smoothness_darkfield": list(np.logspace(-5, 1, 10)),
                                    "sparse_cost_darkfield": list(np.logspace(-5, 1, 10))},
                        init_params={"smoothness_flatfield": 0.1,
                                    "smoothness_darkfield": 1e-5,
                                    "sparse_cost_darkfield": 1e-5})               
        # basic_2 = BaSiC.load_model(os.path.join(dest, "basic_second"))
        images_transformed = basic_2.fit_transform(images_transformed_ff)
        basic_2.save_model(os.path.join(dest, "basic_second"), overwrite=True)
        # print(f"Done Illumination Correction cyc{str(cycles).zfill(2)} Z0{str(zplanes).zfill(2)}_CH{str(channels)}")
        
    else: 
        
        print(f"Start Illumination Correction cyc{str(cycles).zfill(2)} Z0{str(zplanes).zfill(2)}_CH{str(channels)}")
        basic = BaSiC.load_model(os.path.join(dest, "basic_first"))
        # ratio_2 = 1 / np.abs(8-zplanes)
        # fitting_2 = basic.flatfield * ratio_2 * np.ones_like(im_array)
        images_transformed_ff2 = basic.fit_transform(im_array)
        
        # print(f"Image quantile: {np.quantile(im_array, quant)} flatfield quantile: {np.quantile(basic.flatfield, quant)}")
        # print(f"Cycle {cycles} channel {channels} z-plane {zplanes} first ratio: {ratio_1}")
        # if 0 > ratio_1:
        #     print("Negative ratio. Multiplying by -1.")
        #     ratio_1 = ratio_1 * -1
        #     fitting_1 = basic.flatfield * ratio_1 * np.ones_like(im_array)
        #     images_transformed_ff2 = basic.fit_transform(im_array, fitting_1)
        # if np.isinf(ratio_1):
        #     print("Infinite ratio. Using no first fitting weight.")
        #     # fitting_1 = basic.flatfield / ratio_1 * np.ones_like(im_array)
        #     images_transformed_ff2 = basic.fit_transform(im_array)
        # else:
        #     print("Using first multiplier fitting weight.")
        #     fitting_1 = basic.flatfield * ratio_1 * np.ones_like(im_array)
        #     images_transformed_ff2 = basic.fit_transform(im_array, fitting_1)
        
        basic_3 = BaSiC.load_model(os.path.join(dest, "basic_second"))
        # basic_3.autotune(im_array, random_state=0, histogram_bins=50, histogram_qmin=1-quant, histogram_qmax=quant, 
        #                 early_stop_n_iter_no_change=5, early_stop_tolerance=0.001,
        #                 search_space={"smoothness_flatfield": list(np.logspace(-1, 1, 10)),
        #                             "smoothness_darkfield": list(np.logspace(-5, 1, 10)),
        #                             "sparse_cost_darkfield": list(np.logspace(-5, 1, 10))},
        #                 init_params={"smoothness_flatfield": basic_3.settings.get("smoothness_flatfield"),
        #                             "smoothness_darkfield": basic_3.settings.get("smoothness_darkfield"),
        #                             "sparse_cost_darkfield": basic_3.settings.get("sparse_cost_darkfield")})
        # ratio_3 = 1 / np.abs(8-zplanes)
        # fitting_3 = basic.darkfield * ratio_3 * np.ones_like(images_transformed_ff2)
        images_transformed = basic_3.fit_transform(images_transformed_ff2)
        # # print(f"Image quantile: {np.quantile(images_transformed_ff2, 1-quant)} darkfield quantile: {np.quantile(basic_2.darkfield, 1-quant)}")
        # print(f"Cycle {cycles} channel {channels} z-plane {zplanes} second ratio: {ratio_2}")
        # if 0 > ratio_2:
        #     print("Negative ratio. Multiplying by -1.")
        #     ratio_2 = ratio_2 * -1
        #     fitting_2 = basic_2.darkfield * ratio_2 * np.ones_like(images_transformed_ff2)
        #     images_transformed = basic_2.fit_transform(images_transformed_ff2, fitting_2)
        # if np.isinf(ratio_2):
        #     print("Infinite ratio. Using no second fitting weight.")
        #     images_transformed = basic_2.fit_transform(images_transformed_ff2)
        # else:
        #     print("Using second multiplier fitting weight.")
        #     fitting_2 = basic_2.darkfield * ratio_2 * np.ones_like(images_transformed_ff2)
        #     images_transformed = basic_2.fit_transform(images_transformed_ff2, fitting_2)
       
        print(f"Done Illumination Correction cyc{str(cycles).zfill(2)} Z0{str(zplanes).zfill(2)}_CH{str(channels)}")

    im_t_max, im_t_min = np.max(images_transformed), np.min(images_transformed)
    print(f"Cycle {cycles} channel {channels} z-plane {zplanes} min, max is {im_t_min}, {im_t_max} range is {im_t_max - im_t_min}")
    if np.min(images_transformed) != 0:
        images_transformed = images_transformed - np.min(images_transformed)   
    images_transformed = exposure.match_histograms(images_transformed, im_array).astype(np.uint16) 
    # images_transformed = exposure.rescale_intensity(images_transformed, out_range=(0, 65535)).astype(np.uint16)
    # images_transformed = exposure.equalize_adapthist(images_transformed, kernel_size=500, clip_limit=0.0, nbins=100).astype(np.uint16)
                                              
    stitch(images_transformed, zplanes, dest, dest_1, channels, zplanes_n, pou, rows, cols, overlap_percentage)
    
    

### 3.3 Running multiple cycle/channels

This cell runs the above two functions for all the images you define with start_cycle, end_cycle, start_channel, and end_channel.  You must enter these values as well as n, m, pou (determined from the first notebook), zplanes_n (number of zplanes), and overlap_percentage.  You must also enter the number of workers for the multithreading (usually determined by dataset size, available memory, and number of cores).  See https://docs.python.org/3/library/concurrent.futures.html

To run just one cycle or channel, simply enter the same number for both start and end.

If problems with the jupyter kernel restarting becomes an issue, decrease the number of workers, or close other programs to conserve resources.  You can also try reducing the histogram bins, number of iterations, and/or early_stop_n_iter_no_change values.

In [6]:
n = 9  # Number of rows (height)
m = 7  # Number of columns (width)

# Row coordinates: each row index is repeated m times
rows = list(chain.from_iterable(repeat(row, m) for row in range(n)))
# Column coordinates: snake pattern for each row, going back and forth from top left going right
cols = list(chain.from_iterable(range(m) if row % 2 == 0 else range(m - 1, -1, -1) for row in range(n)))

start_cycle = 1
end_cycle = 1
start_channel = 4
end_channel = 4
pou = 13
zplanes_n = 17
overlap_percentage = 0.30
workers = (zplanes_n-1)//2

image_dir_list = [image_dir] * (zplanes_n-1)
stitch_dir_list = [stitch_dir] * (zplanes_n-1)
zplanes_list =[zplanes_n] * (zplanes_n-1)
pou_list = [pou] * (zplanes_n-1)
rows_list = [rows] * (zplanes_n-1)
cols_list = [cols] * (zplanes_n-1)
overlap_percentage_list = [overlap_percentage] * (zplanes_n-1)


for j in range(start_cycle, end_cycle+1):
    for i in range(start_channel, end_channel+1):

        cycles = j
        zplanes = zplanes_n//2 
        channels = i
        apply_basic(image_dir, stitch_dir, zplanes, cycles, channels, zplanes_n, pou, rows, cols, overlap_percentage)
        
        if __name__ ==  '__main__':
            with ThreadPoolExecutor(max_workers=workers) as executor:
                cycles = [j] * (zplanes_n-1) 
                zplanes =  list(range(1, zplanes_n//2))+list(range((zplanes_n//2)+1, zplanes_n+1)) 
                channels = [i] * (zplanes_n-1)
                executor.map(apply_basic, image_dir_list, stitch_dir_list, zplanes, cycles, channels, zplanes_list, pou_list, rows_list, cols_list, overlap_percentage_list) 
        

vmin_init: 1735.0 vmax_init: 31877.0 val_range_init: 46254.0
vmin: 1749.19580078125 vmax: 31933.834082031215 val_range: 46326.4749023437


[0] fit_and_calc_entropy (Hill Climbing):   0%|          | 0/100 [00:00<?, ?it/s]

vmin_new: 1049.51748046875 


[0] fit_and_calc_entropy (Hill Climbing):   2%|          | 2/100 [00:06<10:48,  6.62s/it, best_iter=0, best_pos=[0], best_score=-127.99927837293735]

Entropy: 9.64912489850509, Fourier L0 norm: 118.35015347443226
vmin_new: 1057.731884765625 


...

[0] fit_and_calc_entropy (Hill Climbing):  32%|───       | 16/50 [00:46<01:36,  2.85s/it, best_iter=0, best_pos=[0 0 1], best_score=-1e-15]

Entropy: 8.482837276526698, Fourier L0 norm: 44.39017995402286
vmin_new: 375.1192745361328 


[0] fit_and_calc_entropy (Hill Climbing):  32%|───       | 16/50 [00:51<01:48,  3.20s/it, best_iter=0, best_pos=[0 0 1], best_score=-1e-15]

Entropy: 8.47587861069215, Fourier L0 norm: 43.24437272486284


Results: 'fit_and_calc_entropy'  
   Best score: -1e-15  
   Best parameter set:
      'smoothness_flatfield'  : 0.1  
      'smoothness_darkfield'  : 1e-05  
      'sparse_cost_darkfield' : 4.641588833612782e-05  
   Best iteration: 0  
 
   Random seed: 0  
 
   Evaluation time   : 51.05472135543823 sec    [100.0 %]
   Optimization time : 0.0012307167053222656 sec    [0.0 %]
   Iteration time    : 51.055952072143555 sec    [1.02 sec/iter]
 


Cycle 1 channel 4 z-plane 8 min, max is -2615.69677734375, 98045.171875 range is 100660.8671875
Start Illumination Correction cyc01 Z001_CH4
Start Illumination Correction cyc01 Z002_CH4
Start Illumination Correction cyc01 Z006_CH4
Start Illumination Correction cyc01 Z003_CH4
Start Illumination Correction cyc01 Z004_CH4
Start Illumination Correction cyc01 Z005_CH4
Start Illumination Correction cyc01 Z007_CH4
Start Illumination Correction cyc01 Z009_CH4
Done Illumination Correction cyc01 Z001_CH4
Cycle 1 channel 4 z-plane 1 min, max is -3026.767822265625, 59173.0546875 range is 62199.82421875
Done Illumination Correction cyc01 Z002_CH4
Cycle 1 channel 4 z-plane 2 min, max is -3259.625732421875, 63637.62109375 range is 66897.25
Done Illumination Correction cyc01 Z003_CH4Done Illumination Correction cyc01 Z007_CH4

Cycle 1 channel 4 z-plane 7 min, max is -2342.645751953125, 100532.0703125 range is 102874.71875
Cycle 1 channel 4 z-plane 3 min, max is -3405.1865234375, 72059.4140625 range is

### 4. Deconvolution

Before moving on to the deconvolution step, it is a good idea to review the results thus far.  If they are satisfactory, the parameters for deconvolution discovered in the first notebook can be entered below to define the decon function. Also, if more or all of the cycles have now been stitched, more deconvolution testing can be done on these stacks in the previous notebook before proceeding to a larger run. 

Be sure the MATLAB Runtime v9.5 (R2018, 64-bit) is installed.  Download for free here: http://www.mathworks.com/products/compiler/mcr/index.html. Reboot your computer after install.  

There is no need to supply a point-spread function.

The base_dir and stitch_dir definitions used previously will be again used here, and a new folder will be created to save the deconvolved images.  

The original deconvolution program was written specifically for lightsheet images. Here the estimation of the point-spread function was modified for use with widefield fluorescence images.  See https://www.nature.com/articles/s41598-019-53875-y for the original publication which includes links to the original code.

Output will be written to the terminal.  Check that the first image stack is processed without error.  If there is an "Maximum variable size allowed on the device is exceeded." error, then restart the kernel, decrease the max_GPU or max_CPU parameter, rerun the decon function cell, and try again.

### 4.1 Deconvolution Function

In [ ]:
def decon(cycles, channels):
    
    c = str(cycles)
    ch = str(channels)

    # pixel size in xy dimension (nanometers)
    xy_vox = 377
    # pixel size in z dimension (nanometers)
    z_vox = 1500
    # Number of iterations of Lucy-Richardson algo before stopping unless stop_crit is met first
    iterations = 100
    # Microscope objective numerical aperture
    mic_NA = 0.75
    # Refractive index of tissue being imaged
    tissue_RI = 1.3
    # Opening size in millimeters of objective aperture
    slit_aper = 6.5
    # Focal length in millimeters of objective
    f_cyl = 1
    # Used to reduce noise.  Increase value for noisy images. (0-10)
    damping = 0
    # If set, the deconvolved images will be clipped by this percent for max and min values, and then scaled to full range of bit depth. (0-5)
    hist_clip = 0.000
    # Percent change between iterations to use as criteria to stop deconvolution.
    stop_crit = 5.00
    # Enter 1 to perform on GPU, 0 to use CPU
    GPU = 1
    # Percent maximum GPU memory to use if GPU = 1
    max_GPU = 25
    # Percent maximum RAM to use if GPU = 0
    max_CPU = 40
    if GPU == 1:
        max_block=max_GPU
    elif GPU == 0:
        max_block=max_CPU

    # The respective excitation and emission wavelength in nanometers for each channel
    C1ex = 358
    C1em = 461
    C2ex = 753
    C2em = 775
    C3ex = 560
    C3em = 575
    C4ex = 648
    C4em = 668
    
    # The path to the deconvolution executable file.
    decon_exe = "C:/Users/smith6jt/KINTSUGI/LsDeconv.exe"
    stitch_dir = "C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_BaSiC_Stitched"
    decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
    source = os.path.join(stitch_dir, f"cyc{c.zfill(2)}", f"CH{ch}")
    dest = os.path.join(decon_dir, f"cyc{c.zfill(2)}", f"CH{ch}")
    os.makedirs(dest, exist_ok=True)

    print(f"Starting Deconvolution of cyc{c.zfill(2)} CH{ch}")
   
    if channels==1:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C1ex), str(C1em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
        # print(result_1.stdout)
    if channels==2:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C2ex), str(C2em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==3:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C3ex), str(C3em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    if channels==4:
        subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(C4ex), str(C4em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
    # gc.collect()
    
    try:
        os.rename(os.path.join(source, 'deconvolved'), os.path.join(dest, 'deconvolved'))
    except (FileNotFoundError):
        print("Reduce max memory.")


To apply the above decon function to multiple cycles/channels, enter the start and end numbers below.

In [ ]:
decon_start_cycle = 2
decon_end_cycle = 13
decon_start_channel = 1
decon_end_channel = 4


for j in range(decon_start_cycle, decon_end_cycle+1):
    
    for i in range(decon_start_channel, decon_end_channel+1):
        
        
        decon( j, i)

### 5. Extended Depth of Field

In [ ]:
channel_name_dict = {"cyc01.tif" : ["DAPI", "Blank1a", "Blank1b", "Blank1c"],
 "cyc02.tif" : ["DAPI", "CD31", "CD8", "Empty2c"],
 "cyc03.tif" : ["DAPI", "CD20", "Ki67", "CD3e"],
 "cyc04.tif" : ["DAPI", "SMActin", "Podoplanin", "CD68"],
 "cyc05.tif" : ["DAPI", "PanCK", "CD21", "CD4"],
 "cyc06.tif" : ["DAPI", "Lyve1", "CD45RO", "CD11c"],
 "cyc07.tif" : ["DAPI", "CD35", "ECAD", "CD107a"],
 "cyc08.tif" : ["DAPI", "CD34", "CD44", "HLADR"],
 "cyc09.tif" : ["DAPI", "Empty9a", "FoxP3", "CD163"],
 "cyc10.tif" : ["DAPI", "Empty10a", "CollagenIV", "Vimentin"],
 "cyc11.tif" : ["DAPI", "Empty11a", "CD15", "CD45"],
 "cyc12.tif" : ["DAPI", "Empty12a", "CD5", "CD1c"],
 "cyc13.tif" : ["DAPI", "Blank13a", "Blank13b", "Blank13c"]}

In [ ]:
import imagej, scyjava
ij = imagej.init('C:/Users/smith6jt/Fiji.app', add_legacy=True); print(ij.getVersion())

In [ ]:
import imagej, scyjava
ij_mem = 40
GPU_name = "NVIDIA RTX A4500"
radius_x = 5.0
radius_y = 5.0
sigma = 20.0
edf_start_cycle = 1
edf_end_cycle = 13
edf_start_channel = 1
edf_end_channel = 4
scyjava.config.add_option(f'-Xmx{str(ij_mem)}g')
ij = imagej.init('C:/Users/smith6jt/Fiji.app', add_legacy=True)
decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
edf_dir = decon_dir.replace('_Decon', '_EDF')

macro = """
#@ File in_folder
#@ String device
#@ File out_folder
#@ String file_name
#@ Integer radius_x
#@ Integer radius_y
#@ Integer sigma

File.openSequence(in_folder);
run("CLIJ2 Macro Extensions", "cl_device=[" + device + "]");
Ext.CLIJ_clear();
image1 = "deconvolved";
Ext.CLIJ2_push(image1);
image2 = "extended_depth_of_focus_variance_projection";
radius_x = 5.0;
radius_y = 5.0;
sigma = 20.0;
Ext.CLIJ2_extendedDepthOfFocusVarianceProjection(image1, image2, radius_x, radius_y, sigma);
Ext.CLIJ2_pull(image2);
selectImage(image1);
close();
selectImage(image2);
saveAs("Tiff", out_folder + File.separator + file_name);
"""

for j in range(edf_start_cycle, edf_end_cycle+1):
    
    for i in range(edf_start_channel, edf_end_channel+1):

        edf_source = os.path.join(decon_dir, f"cyc{str(j).zfill(2)}", f"CH{str(i)}", "deconvolved")
        edf_dest = os.path.join(edf_dir, f"cyc{str(j).zfill(2)}")
        file_name = channel_name_dict.get(f"cyc{str(j).zfill(2)}.tif")[i-1]
        os.makedirs(edf_dest, exist_ok=True)

        args ={'in_folder': edf_source, "cl_device" : GPU_name, "out_folder" : edf_dest, "file_name" : file_name, "radius_x" : 5.0, "radius_y" : 5.0, "sigma" : 20.0}
        ij.py.run_macro(macro, args)

In [ ]:
import math
import logging

logging.basicConfig(level=logging.WARN) #change to INFO for more details
# Add vips binaries to path
vipshome = 'C:/Users/smith6jt/KINTSUGI/vips-dev-8.16/bin'
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips 
# logging.info("vips version: " + str(pyvips.version(0))+"."+str(pyvips.version(1))+"."+str(pyvips.version(2)))


idx = list(range(1, 14))
imagedir = 'C:/Users/smith6jt/KINTSUGI/data/1904_CC2B28_EDF'
outputdir = imagedir #change this to save to a different folder

dataSets = glob.glob1(imagedir, f'/cyc{str(idx).zfill(2)}/*.tif')

def eval_cb(image, progress):
        pbar_filesave.update(progress.percent - pbar_filesave.n)


for idx, dataset in enumerate(dataSets):
    str1 = dataset.split(".tif", 2)
    #print("Processing: "+ str1[0] + " " + str(idx) + " out of " + str(len(dataSets)))
    imagePrefix = str1[0]
    imageFiles = glob.glob(os.path.join(imagedir, imagePrefix+'.tif'))

    outfilename = imagePrefix + '.ome.tif'
    outfile = os.path.join(outputdir, outfilename)
    pbar_filesave = tqdm(total=100, unit="Percent", desc=outfilename, position=0, leave=True)

    images = [pyvips.Image.new_from_file(os.path.join(imagedir, filename), access="sequential") for filename in imageFiles]

    out = pyvips.Image.arrayjoin(images, across=1)
    out = out.copy()
    out.set_type(pyvips.GValue.gint_type, "page-height", images[0].height)

    # Parameters for OME metadata
    width = images[0].width
    height = images[0].height

    # Need to handle more data types
    data_type = "uint16"
    if (images[0].interpretation=="grey16"):
        data_type = "uint16"
        
    bands = int(out.height/images[0].height)
    CalibrationX = 1000/float(out.xres) #Or out.xres
    CalibrationY = 1000/float(out.yres) #Or out.yres
    CalibrationUnits = "µm" #Or out.get("resolution-unit")
    channel_names = channel_name_dict.get(f"cyc{str(idx).zfill(2)}.tif")
    
    # build minimal OME metadata
    metadata = f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="{bands}"
                    SizeT="1"
                    SizeX="{width}"
                    SizeY="{height}"
                    SizeZ="1"
                    Type="{data_type}"
                    PhysicalSizeX="{CalibrationX}"
                    PhysicalSizeXUnit="{CalibrationUnits}"
                    PhysicalSizeY="{CalibrationY}"
                    PhysicalSizeYUnit="{CalibrationUnits}">
                <Channel ID="Channel:0" Color="16711935" Name={channel_names[0]} SamplesPerPixel="1" />
                <Channel ID="Channel:1" Color="-1208024833" Name={channel_names[1]} SamplesPerPixel="1" />
                <Channel ID="Channel:2" Color="1828651263" Name={channel_names[2]} SamplesPerPixel="1" />
                <Channel ID="Channel:3" Color="11206655" Name={channel_names[3]} SamplesPerPixel="1" />
            </Pixels>
        </Image>
    </OME>"""

    out.set_type(pyvips.GValue.gstr_type, "image-description", metadata)
    out.set_progress(True)
    out.signal_connect('eval', eval_cb)
    
    out.write_to_file(outfile, compression='lzw', subifd=True, bigtiff=True)